# Initialization

In [1]:
# Parameters. 
# Times Square replaces this cell with the user's parameters.

# The run-time defaults for all of these parameters are in NightLog.yaml
# Under Times Square, the run-time defaults always override values given here.
# Values here are used for local tests.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-18' # Value to use for local testing (Summit)
day_obs = 'TODAY' # TODO Change to 'TODAY' to test with default before push  

# Total number of days of data to display (ending on day_obs)
number_of_days = '3'  # TODO Change to '1' to test with default before push  

In [2]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp
from urllib.parse import urlencode
from IPython.display import display, Markdown, display_markdown
from matplotlib import pyplot as plt
import os
from datetime import datetime, date, timedelta
#! from rubin_scheduler.site_models import Almanac

# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 50  # YAGNI: Auto get more if this isn't enough to get all requested DAYS

match day_obs.lower():
    case 'today':
        date = datetime.now().date()
    case 'yesterday':
        date = datetime.now().date()-timedelta(days=1)
    case _:
        date = datetime.strptime(day_obs, '%Y-%m-%d').date()
# date:  is EXLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
min_day_obs = (date - timedelta(days=days-1)).strftime('%Y%m%d') # Inclusive
max_day_obs = (date + timedelta(days=1)).strftime('%Y%m%d') # prep for Exclusive

response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get('EXTERNAL_INSTANCE_URL', summit) # TODO try with "usdf" before push (else "summit")

In [4]:
# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get('EXTERNAL_INSTANCE_URL'):
    print('Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....')
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype >/dev/null
import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.almanac as alm
import lsst.ts.logging_and_reporting.reports as rep 
from lsst.ts.logging_and_reporting.reports import md,mdlist, NightlyLogReport

try:
    import lsst.ts.logging_and_reporting.version
    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = 'LIVE'

try:
    from lsst_efd_client import EfdClient
    enable_efd = True
except:
    enable_efd = False

# Overview

The only environment that has everything needed for this page is
https://summit-lsp.lsst.codes

However, Times Square **does not** run on the Summit. It **does** run on USDF-dev.  USDF doesn't fully support all the data sources we need so some functionality is currently missing on this page.

In [5]:
# Display overview of Report context 
md(f'''
Report for **{date}** covering the previous **{days}** observing night(s).
- Run on logs from **{server}/**
- Using *Prototype* Logging and Reporting Version: **{lrversion}**
''')

ul = '\n- '.join(['',*sad.all_endpoints(server)])
md(f'This report will attempt to use the following log sources: {ul}')


Report for **2024-09-18** covering the previous **3** observing night(s).
- Run on logs from **https://summit-lsp.lsst.codes/**
- Using *Prototype* Logging and Reporting Version: **0.1.dev63+g83a61e0.d20240918**


This report will attempt to use the following log sources: 
- https://summit-lsp.lsst.codes/exposurelog/instruments
- https://summit-lsp.lsst.codes/exposurelog/exposures
- https://summit-lsp.lsst.codes/exposurelog/messages
- https://summit-lsp.lsst.codes/narrativelog/messages
- https://summit-lsp.lsst.codes/nightreport/reports

In [6]:
print()

# Almanac

In [7]:
# Display various almanac values (for moon, sun)
rep.AlmanacReport().almanac_as_dataframe()

,0,1
Moon Rise,2024-09-17 22:25:43.601,
Moon Set,2024-09-17 10:17:44.848,
Moon Illumination,99%,(% lit)
Astronomical Twilight (morning),2024-09-17 09:18:25.687,(-18 degrees)
Astronomical Twilight (evening),2024-09-17 23:56:37.079,(-18 degrees)
Nautical Twilight (morning),2024-09-17 09:46:22.573,(-12 degrees)
Nautical Twilight (evening),2024-09-17 23:28:37.050,(-12 degrees)
Civil Twilight (morning),2024-09-17 10:14:10.623,(-6 degrees)
Civil Twilight (evening),2024-09-17 23:00:46.504,(-6 degrees)
Sun Rise,2024-09-17 10:41:56.478,


# Night Report

In [8]:
# Get data from Night Report log. Display nightly Jira BLOCKS.
nr_adapter = sad.NightReportAdapter(server_url=server,
                                    limit=limit,
                                    min_day_obs=min_day_obs,
                                    max_day_obs=max_day_obs,)
nr_url = nr_adapter.source_url
try:
    nr_recs,nr_url = nr_adapter.get_reports()
except Exception as err:
    nr_recs = []
    msg = f'ERROR getting records from {nr_url=}: {err=}'
    raise Exception(msg)

# print(f'Retrieved {len(nr_recs)} records from {nr_url}')

# Display Jira BLOCKS
front = 'https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/'
tickets = nr_adapter.nightly_tickets(nr_recs)

if tickets:
    mdstr = '## Nightly Jira BLOCKs'
    for day, url_list in tickets.items():
        mdstr += f'\n- {day}'
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    md(f'No jira BLOCK tickets found.', color='lightblue')
    md(f'Used: [API Data]({nr_url})')

# Display time log
nr_rep = NightlyLogReport(min_day_obs=min_day_obs, max_day_obs=max_day_obs)
nr_rep.time_log_as_markdown(nr_recs, nr_adapter, nr_url)
    

## Nightly Jira BLOCKs
- 20240916
    - [testPlayer/BLOCK-R89](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testPlayer/BLOCK-R89)
    - [v2/testCases](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCases)
- 20240917
    - [v2/testCase/BLOCK-T19](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/BLOCK-T19)
    - [testPlayer/BLOCK-R90](https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/testPlayer/BLOCK-R90)

### 

## NIGHT: 20240916: 

### DATE: 2024-09-16: 

> 12:18:03 | <pre>Daytime: Cycle 39 started around 09:00 CLT, so most of CSCs were in Offline or Standby until it was done. 

We run successfully all the blocks. 
- BLOCK-T151 (1.0) Simonyi Hexapods Startup. Both M2/Camera Hexapods shows 17.3 C.
- BLOCK-T3 (2.0) Warm-up M2 Hexapod.
- BLOCK-T4 (2.0) Warm-up Camera Hexapod.
- BLOCK-T153 - Opening M1M3 Mirror Covers.
- BLOCK-T152  - Closing M1M3 Mirror Covers.
- BLOCK-T145 - M1M3 Hardpoints Breakaway Test.
- BLOCK-T144 - M1M3 Bump Test - Actuators [238, 330, 409] FAILED the bump test.
- BLOCK-T139 - Hexapod Shutdown.

The ScriptQueue:1 was updated and patched during the night. After the warm-up, M2 and Camera Hexapod were in disabled state during all the run and we monitor their temperature. Since the end of the warm-up until the hexapod shutdown their temperature increased ~ 1.5 deg C (average).</pre>

...

> 12:46:16 | <pre>During daytime, Petr K. worked on M1M3 to run bump tests, raise the mirror, and test to find the reference/center position for static support centering (see more details and results at #m1m3_worklog channel). 

We started from Simonyi Hexapods Startup (BLOCK-T151) around 18:30 UTC and it was done without any issues. Then warm-up M2 Hexapod (BLOCK-T3) and warm Camera Hexapod (BLOCK-T4) were completed successfully. Both components were left at enabled state and were monitored.

Camera Hexapod Strut12 showed a notorious increase on temperature (about 2C in about 20 min plus diverting in motor current from the rest of the struts). It was turned off and left in Standby state. 

M2 Hexapod, we kept monitoring its currents (as temperature sensors is not available)




</pre>

> 12:46:16 | <pre>During daytime, Petr K. worked on M1M3 to run bump tests, raise the mirror, and test to find the reference/center position for static support centering (see more details and results at #m1m3_worklog channel). 

We started the Simonyi Hexapods Startup (BLOCK-T151) around 18:30 UTC and it was done without any issues. Then warm-up M2 Hexapod (BLOCK-T3) and warm Camera Hexapod (BLOCK-T4) were completed successfully. Both components were left at enabled state and were monitored (see OBS-592).

Camera Hexapod Strut12 showed a notorious increase on temperature (about 2C in about 20 min plus diverting in motor current from the rest of the struts). It was turned off and left in standby state. 

For M2 Hexapod, we kept monitoring its currents on enabled state (as temperature sensors are not available, OBS-593). At about 23:30 CLT, motor current on leg 4 approached the 4 amp. We proceed to shutdown the hexapod but it was not possible (either locally nor remote) to access the PDU with the known credentials (IHS-8426).

The remaining Simonyi tests (BLOCK-T145 M1M3 Hardpoints Breakaway Test and BLOCK-T144 M1M3 Bump Test) run successfully. Actuators [218, 238, 409] FAILED the bump test. 



</pre>

> 12:46:16 | <pre></pre>

> 12:46:16 | <pre>During daytime, Petr K. worked on M1M3 to run bump tests, raise the mirror, and test to find the reference/center position for static support centering (see more details and results at #m1m3_worklog channel). 

We started from Simonyi Hexapods Startup (BLOCK-T151) around 18:30 UTC and it was done without any issues. Then warm-up M2 Hexapod (BLOCK-T3) and warm Camera Hexapod (BLOCK-T4) were completed successfully. 

</pre>

:EOT

# Exposure Log

In [9]:
# Get data from Exposure log. Display time log.
exposure_adapter = sad.ExposurelogAdapter(
    server_url=server,
    limit=limit,
    min_day_obs=min_day_obs,
    max_day_obs=max_day_obs,
)
exposure_url = exposure_adapter.source_url
try:
    exposure_recs,url = exposure_adapter.get_messages()
except Exception as err:
    exposure_recs = []
    msg = f'ERROR getting records from {url=}: {err=}'
    raise Exception(msg)

if exposure_recs:
    table = exposure_adapter.day_table(exposure_recs,'date_added', dayobs_field='day_obs')
    #print(table)
    mdlist(table)
else:
    md(f'No Exposure Log records found.', color='lightblue')
    md(f'Used [API Data]({exposure_url})')

### <font color='lightblue'>No Exposure Log records found.</font>

Used [API Data](https://summit-lsp.lsst.codes/exposurelog)

In [10]:
len(exposure_recs)

0

In [11]:
# Display Observation gaps
if usdf == os.environ.get('EXTERNAL_INSTANCE_URL'):
    md(f"**Warning:** The `/exposures/` endpoint is not yet functional on SERVER=usdf.", color='red')
gaps = exposure_adapter.get_observation_gaps()
if gaps:
    md(f'### Date vs Observation Gap (minutes) for all Instruments')
    for instrument, day_gaps in gaps.items():
        if len(day_gaps) == 0:
            md(f'**No day gaps found for *{instrument=!s}* **', color='lightblue')
        else:
            x,y = zip(*day_gaps.items())
            df = pd.DataFrame(dict(day=x,minutes=y))
            df.plot.bar(x='day', y='minutes', title=f'{instrument=!s}')
else:
    md(f'No Observation Gaps found in exposures.', color='lightblue')

### <font color='lightblue'>No Observation Gaps found in exposures.</font>

# Narrative Log


In [12]:
# Get data from Narrative log. Display time log.
narrative_adapter = sad.NarrativelogAdapter(
    server_url=server,
    limit=limit,
    min_day_obs=min_day_obs,
    max_day_obs=max_day_obs,
)
narrative_url = narrative_adapter.source_url
try:
    # date like '2000-01-02 12:00:00'
    # str(datetime(2000, 1, 2, 12, 0, 0))
    min_date = str(datetime.strptime(min_day_obs,'%Y%m%d'))
    max_date = str(datetime.strptime(max_day_obs,'%Y%m%d'))
    #!print(f'Get data from {narrative_url}: {min_date} to {max_date}')
    narrative_recs,url = narrative_adapter.get_messages()
except Exception as err:
    narrative_recs = []
    msg = f'ERROR getting records from {url}: {err=}'
    raise Exception(msg)

# print(f'Retrieved {len(narrative_recs)} records.')

if narrative_recs:
    md('Warning: Some text of Narrative log message may confuse markdown rendering.',
      color='mediumblue')
    table = narrative_adapter.day_table(narrative_recs, 'date_added')
    #print(tabstr)
    #mdlist(table, color="darkblue")
    mdlist(table)
else:
    md(f'No Narrative Log records found.', color='lightblue')
    md(f'Used [API Data]({narrative_url})')

### <font color='mediumblue'>Warning: Some text of Narrative log message may confuse markdown rendering.</font>

## NIGHT: 20240916: 

### DATE: 2024-09-17: 

> 02:05:26 | <pre><code>Resumed AuxTel on-sky
</code></pre>

> 02:31:51 | <pre><code>Output of the Bump Test:
2024/09/17 02:26:20 TAI


RuntimeError: Actuators [238, 330, 409] FAILED the bump test.


SAA (Single Actuator Axes) Failures:
  None
DAA (Dual Actuator Axes) Failures:
  - Actuator ID 238: Pri Index 72
  - Actuator ID 238: Sec Index 55
  - Actuator ID 330: Pri Index 107
  - Actuator ID 330: Pri Index 107
  - Actuator ID 409: Pri Index 123
  - Actuator ID 409: Pri Index 123






</code></pre>

...

> 03:38:43 | <pre><code>M2 Hexapod set to disabled state. 
Currents now are around 0 
</code></pre>

> 04:41:06 | <pre><code>Stopping observations and will begin shutdown
</code></pre>

> 04:53:33 | <pre><code>AuxTel Dome and Telescope at parking position.
ATTCS and LATISS components in Standby.
</code></pre>

> 04:54:04 | <pre><code>Final status: 
MTMount, M1M3 disabled.
MTPtg &amp; MTM2 enabled.
Camera Hexapod in Standby state with drives and PXI turned off.
M2 Hexapod in Disabled state with drives and PXI turned on.
</code></pre>

:EOT

In [13]:
# Conditionally display our current ability to connect to all needed endpoints.
if not os.environ.get('EXTERNAL_INSTANCE_URL'):
    md('# Dashboard')
    md('(This is not done when running under Times Square.)')
    %run ./dashboard.ipynb

# Dashboard

(This is not done when running under Times Square.)

Try connect to each endpoint of https://summit-lsp.lsst.codes/exposurelog 
Try connect to each endpoint of https://summit-lsp.lsst.codes/narrativelog 
Try connect to each endpoint of https://summit-lsp.lsst.codes/nightreport 
Try connect to each endpoint of https://usdf-rsp-dev.slac.stanford.edu/exposurelog 
Try connect to each endpoint of https://usdf-rsp-dev.slac.stanford.edu/narrativelog 
Try connect to each endpoint of https://usdf-rsp-dev.slac.stanford.edu/nightreport 
Try connect to each endpoint of https://tucson-teststand.lsst.codes/exposurelog 
Try connect to each endpoint of https://tucson-teststand.lsst.codes/narrativelog 
Try connect to each endpoint of https://tucson-teststand.lsst.codes/nightreport 

Connected to 9 out of 15 endpoints.(60%)
Successful connects (9): 
	https://summit-lsp.lsst.codes/exposurelog/instruments
	https://summit-lsp.lsst.codes/exposurelog/exposures?instrument=na
	https://summit-lsp.lsst.codes/exposurelog/messages
	https://summit-lsp.lsst.codes/narr

In [14]:
print(f'Finished {str(datetime.now())}')

Finished 2024-09-18 14:53:49.364756


In [16]:
print(os.getcwd())
print(os.listdir())
%run consdb/assorted_plots.ipynb

/Users/vbecker/Develop/lsst/ts_repos/ts_logging_and_reporting/notebooks_tsqr
['NightLog.ipynb', 'logrep_proto_1.ipynb', 'requirements.txt', 'NightLog.yaml', 'efd.ipynb', 'NightLog2.ipynb', 'scaffolding.org', 'README.md', 'dashboard.ipynb', 'exposurelog.HIDE_yaml', 'consdb', '.ipynb_checkpoints', 'TEMPLATE_logrep.HIDE_yaml', 'logrep_all_env.ipynb', 'narrativelog.HIDE_yaml', 'efd.HIDE_yaml', 'narrativelog.ipynb', 'exposurelog.ipynb', 'TEMPLATE_logrep.ipynb']
None
Installing "lsst.sitcom.summit.utils" from github using "main" branch....
  Running command git clone --filter=blob:none --quiet https://github.com/lsst-sitcom/summit_utils.git /private/var/folders/s5/114qkzh55t36hvzbb6blzdq80000gp/T/pip-req-build-tpomyp4v
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      error: Multiple top-level packages discovered in a flat-layout: ['ups', 'config', 'pipelines'].
      
      To avoid ac

ModuleNotFoundError: No module named 'lsst.summit'

ModuleNotFoundError: No module named 'lsst.summit'